In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## TextBinding

As an introduction to `y-pydantic`, let's look at a TextBinding. It wraps a `YText` reference, and also needs a reference to the `YDoc` CRDT in order to observe changes made to the `YText`, and make changes itself.

In [2]:
import y_py as Y

doc = Y.YDoc()

<IPython.core.display.Javascript object>

In [3]:
from y_pydantic import TextBinding

binding = TextBinding(parent_doc=doc, ytext=doc.get_text("text-key"))
binding

<TextBinding >

<IPython.core.display.Javascript object>

In [4]:
# Each y-pydantic Binding has a pydantic Model representing
# its Y shared data type. For a YText, we have a TextModel.
# Each TextModel.items entry will be a TextItem, more on that later.
binding.model

TextModel(items=[], deleted=[])

<IPython.core.display.Javascript object>

In [5]:
binding.ytext

YText()

<IPython.core.display.Javascript object>

In [6]:
# Add some text using the vanilla Y-py document / txn syntax
with doc.begin_transaction() as txn:
    doc.get_text("text-key").extend(txn, "foo")

doc.get_text("text-key")

YText(foo)

<IPython.core.display.Javascript object>

In [7]:
# same way to reference the YText shared data type
binding.ytext

YText(foo)

<IPython.core.display.Javascript object>

In [8]:
# repr of the binding has similar repr to YText
binding

<TextBinding foo>

<IPython.core.display.Javascript object>

In [9]:
# plain str output of YText and binding is always the same
str(binding.ytext) == binding.plain_text

True

<IPython.core.display.Javascript object>

In [10]:
# Now look at the internal Pydantic model for the binding,
# each item is a character in the Text string with optional
# arbitrary attributes associated with that character
binding.model

TextModel(items=[TextItem(value='f', attributes={}), TextItem(value='o', attributes={}), TextItem(value='o', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

In [11]:
# y-pydantic has convenience functions to enter into a txn for you
binding.extend("bar")
binding

<TextBinding foobar>

<IPython.core.display.Javascript object>

In [12]:
doc.get_text("text-key")

YText(foobar)

<IPython.core.display.Javascript object>

In [13]:
# Creating a diff and syncing with other clients is no different
# when using y-pydantic
diff = doc.begin_transaction().diff_v1()
new_doc = Y.YDoc()
new_doc.begin_transaction().apply_v1(diff)
new_doc.get_text("text-key")

YText(foobar)

<IPython.core.display.Javascript object>

### Text attributes

One motivating factor to create `y-pydantic` is that the `y-py` library does not expose any way to access attributes attached to YText objects.

In [14]:
# Print out changes to the YText
def obs_text(event: Y.YTextEvent):
    print(f"YTextEvent: {event}")


ytext = doc.get_text("text-key")
ytext.observe(obs_text)

# Add arbitrary attributes to the first three characters in the text
with doc.begin_transaction() as txn:
    ytext.format(txn, index=0, length=3, attributes={"bold": True})

ytext

YTextEvent: YTextEvent(target=foobar, delta=[{'retain': 3, 'attributes': {'bold': True}}], path=[])


YText(foobar)

<IPython.core.display.Javascript object>

In [15]:
# but there is no way to access those attributes
str(ytext)

'foobar'

<IPython.core.display.Javascript object>

In [16]:
repr(ytext)

'YText(foobar)'

<IPython.core.display.Javascript object>

In [17]:
ytext.to_json()

'"foobar"'

<IPython.core.display.Javascript object>

In [18]:
try:
    vars(ytext)
except Exception as e:
    print(f"error: {e}")

error: vars() argument must have __dict__ attribute


<IPython.core.display.Javascript object>

In [19]:
# The TextBinding observes those YTextEvent changes, including
# application of attributes on characters that are retained or
# attributes included on new inserts.
binding

<TextBinding foobar>

<IPython.core.display.Javascript object>

In [20]:
binding.model

TextModel(items=[TextItem(value='f', attributes={'bold': True}), TextItem(value='o', attributes={'bold': True}), TextItem(value='o', attributes={'bold': True}), TextItem(value='b', attributes={}), TextItem(value='a', attributes={}), TextItem(value='r', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

In [21]:
# Syncing with a new doc that uses a TextBinding would
# pick up those same changes
new2 = Y.YDoc()
new_binding = TextBinding(parent_doc=new2, ytext=new2.get_text("text-key"))

Y.apply_update(new2, Y.encode_state_as_update(doc))
# ^^ alternative syntax:
# new2.begin_transaction().apply_v1(doc.begin_transaction().diff_v1())

new_binding

<TextBinding foobar>

<IPython.core.display.Javascript object>

In [22]:
new_binding.model

TextModel(items=[TextItem(value='f', attributes={'bold': True}), TextItem(value='o', attributes={'bold': True}), TextItem(value='o', attributes={'bold': True}), TextItem(value='b', attributes={}), TextItem(value='a', attributes={}), TextItem(value='r', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

## ArrayBinding

In addition to `TextBinding`, there are `ArrayBinding` and `MapBinding` objects. They are similar to `TextBinding` in that they observe a Y data type and store the changes in an internal Pydantic model. 

There is no "missing feature" covered here like the attributes in `TextBinding`, but `ArrayBinding` and `MapBinding` do offer a benefit of converting data types into relevant `y-pydantic` bindings.

In [23]:
from y_pydantic import ArrayBinding

doc = Y.YDoc()

yarray = doc.get_array("array-key")


def obs_array(event: Y.YArrayEvent):
    print(f"YArrayEvent: {event}")


yarray.observe(obs_array)

binding = ArrayBinding(parent_doc=doc, yarray=yarray)
binding

<ArrayBinding 0>

<IPython.core.display.Javascript object>

In [24]:
binding.append(1)
binding

YArrayEvent: YArrayEvent(target=[1.0], delta=[{'insert': [1.0]}], path=[])


<ArrayBinding 1>

<IPython.core.display.Javascript object>

In [25]:
yarray

YArray([1.0])

<IPython.core.display.Javascript object>

In [26]:
binding.model

ArrayModel(items=[1.0], deleted=[])

<IPython.core.display.Javascript object>

In [27]:
binding.extend(["a", "b", "c"])
binding

YArrayEvent: YArrayEvent(target=[1.0, 'a', 'b', 'c'], delta=[{'retain': 1}, {'insert': ['a', 'b', 'c']}], path=[])


<ArrayBinding 4>

<IPython.core.display.Javascript object>

In [28]:
yarray

YArray([1.0, 'a', 'b', 'c'])

<IPython.core.display.Javascript object>

In [29]:
binding.model

ArrayModel(items=[1.0, 'a', 'b', 'c'], deleted=[])

<IPython.core.display.Javascript object>

In [30]:
binding.insert(0, Y.YText())

YArrayEvent: YArrayEvent(target=['', 1.0, 'a', 'b', 'c'], delta=[{'insert': [YText()]}], path=[])


<IPython.core.display.Javascript object>

In [31]:
yarray

YArray(['', 1.0, 'a', 'b', 'c'])

<IPython.core.display.Javascript object>

In [32]:
binding.model

ArrayModel(items=[<TextBinding >, 1.0, 'a', 'b', 'c'], deleted=[])

<IPython.core.display.Javascript object>

In [33]:
t = binding.model.items[0]
t

<TextBinding >

<IPython.core.display.Javascript object>

In [34]:
t.extend("hello world")

<IPython.core.display.Javascript object>

In [35]:
yarray

YArray(['hello world', 1.0, 'a', 'b', 'c'])

<IPython.core.display.Javascript object>

In [36]:
binding.model

ArrayModel(items=[<TextBinding hello world>, 1.0, 'a', 'b', 'c'], deleted=[])

<IPython.core.display.Javascript object>

In [37]:
t.model

TextModel(items=[TextItem(value='h', attributes={}), TextItem(value='e', attributes={}), TextItem(value='l', attributes={}), TextItem(value='l', attributes={}), TextItem(value='o', attributes={}), TextItem(value=' ', attributes={}), TextItem(value='w', attributes={}), TextItem(value='o', attributes={}), TextItem(value='r', attributes={}), TextItem(value='l', attributes={}), TextItem(value='d', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

In [38]:
# Serializing to dict returns a TextBinding object in the items list
binding.model.dict()

{'items': [<TextBinding hello world>, 1.0, 'a', 'b', 'c'], 'deleted': []}

<IPython.core.display.Javascript object>

In [39]:
# But y-pydantic has custom JSON encoders so that serializing to json
# breaks out any sub-models
binding.model.json()

'{"items": [{"items": [{"value": "h", "attributes": {}}, {"value": "e", "attributes": {}}, {"value": "l", "attributes": {}}, {"value": "l", "attributes": {}}, {"value": "o", "attributes": {}}, {"value": " ", "attributes": {}}, {"value": "w", "attributes": {}}, {"value": "o", "attributes": {}}, {"value": "r", "attributes": {}}, {"value": "l", "attributes": {}}, {"value": "d", "attributes": {}}], "deleted": []}, 1.0, "a", "b", "c"], "deleted": []}'

<IPython.core.display.Javascript object>

In [40]:
from y_pydantic import ArrayModel

# deserialize JSON back to a full model (TODO: model to binding)
ArrayModel.parse_raw(binding.model.json())

ArrayModel(items=[{'items': [{'value': 'h', 'attributes': {}}, {'value': 'e', 'attributes': {}}, {'value': 'l', 'attributes': {}}, {'value': 'l', 'attributes': {}}, {'value': 'o', 'attributes': {}}, {'value': ' ', 'attributes': {}}, {'value': 'w', 'attributes': {}}, {'value': 'o', 'attributes': {}}, {'value': 'r', 'attributes': {}}, {'value': 'l', 'attributes': {}}, {'value': 'd', 'attributes': {}}], 'deleted': []}, 1.0, 'a', 'b', 'c'], deleted=[])

<IPython.core.display.Javascript object>

## Todo App



In [41]:
from typing import Optional


class TodoCRDT:
    def __init__(self):
        self.doc = Y.YDoc()
        yarray = self.doc.get_array("todos")
        self.todos = ArrayBinding(parent_doc=self.doc, yarray=yarray)

        yarray.observe_deep(self.obs)

    def obs(self, event):
        print(event)

    def add_todo(self, text: str) -> TextBinding:
        """
        Add a new Y.YText entry into the todos Y.YArray.
        After it's inserted, populate it with <text>
        """
        ytext = Y.YText()
        self.todos.append(ytext)
        binding = TextBinding(parent_doc=self.doc, ytext=ytext)
        binding.extend("foo")
        return binding

    def snapshot(self, last_sv: Optional[bytes] = None) -> dict:
        """
        Return the current Todo pydantic model and the binary diff of
        the CRDT so other clients can sync. Optionally pass in a last
        state vector to return a serial update as well as "full update"
        """
        response = {
            "model": self.todos.model,
            "full_diff": self.doc.begin_transaction().diff_v1(),
            "sv": self.doc.begin_transaction().state_vector_v1(),
        }
        if last_sv:
            response["last_sv"] = last_sv
            response["partial_diff"] = self.doc.begin_transaction().diff_v1(last_sv)
        return response

    def __repr__(self):
        return f"<Todos {[item.plain_text for item in self.todos.model.items]}>"


crdt = TodoCRDT()
crdt

<Todos []>

<IPython.core.display.Javascript object>

In [42]:
todo = crdt.add_todo("foo")
todo

[YArrayEvent(target=[''], delta=[{'insert': [YText()]}], path=[])]
[YTextEvent(target=foo, delta=[{'insert': 'foo'}], path=[0])]


<TextBinding foo>

<IPython.core.display.Javascript object>

In [43]:
crdt

<Todos ['foo']>

<IPython.core.display.Javascript object>

In [44]:
todo.model

TextModel(items=[TextItem(value='f', attributes={}), TextItem(value='o', attributes={}), TextItem(value='o', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

In [45]:
# Add some attributes to demo a feature that may or may not work down below
# when we sync a new Client to this TodoCRDT
todo.format(index=0, length=2, attributes={"some": "attributes"})

[YTextEvent(target=foo, delta=[{'retain': 2, 'attributes': {'some': 'attributes'}}], path=[0])]


<IPython.core.display.Javascript object>

In [46]:
todo.model

TextModel(items=[TextItem(value='f', attributes={'some': 'attributes'}), TextItem(value='o', attributes={'some': 'attributes'}), TextItem(value='o', attributes={})], deleted=[])

<IPython.core.display.Javascript object>

In [47]:
snapshot = crdt.snapshot()
snapshot

{'model': ArrayModel(items=[<TextBinding foo>], deleted=[]),
 'full_diff': b'\x01\x05\xdf\xc1\xa5\xe9\x0f\x00\x07\x01\x05todos\x02\x04\x00\xdf\xc1\xa5\xe9\x0f\x00\x02fo\x84\xdf\xc1\xa5\xe9\x0f\x02\x01oF\xdf\xc1\xa5\xe9\x0f\x01\x04some\x0c"attributes"\xc6\xdf\xc1\xa5\xe9\x0f\x02\xdf\xc1\xa5\xe9\x0f\x03\x04some\x04null\x00',
 'sv': b'\x01\xdf\xc1\xa5\xe9\x0f\x06'}

<IPython.core.display.Javascript object>

In [48]:
print(snapshot["model"].json(indent=2))

{
  "items": [
    {
      "items": [
        {
          "value": "f",
          "attributes": {
            "some": "attributes"
          }
        },
        {
          "value": "o",
          "attributes": {
            "some": "attributes"
          }
        },
        {
          "value": "o",
          "attributes": {}
        }
      ],
      "deleted": []
    }
  ],
  "deleted": []
}


<IPython.core.display.Javascript object>

In [49]:
# Add another YText/TextBinding to demonstrate loading different snapshots below
crdt.add_todo("bar")
crdt

[YArrayEvent(target=['foo', ''], delta=[{'retain': 1}, {'insert': [YText()]}], path=[])]
[YTextEvent(target=foo, delta=[{'insert': 'foo'}], path=[1])]


<Todos ['foo', 'foo']>

<IPython.core.display.Javascript object>

In [50]:
snapshot2 = crdt.snapshot(last_sv=snapshot["sv"])
snapshot2

{'model': ArrayModel(items=[<TextBinding foo>, <TextBinding foo>], deleted=[]),
 'full_diff': b'\x01\x07\xdf\xc1\xa5\xe9\x0f\x00\x07\x01\x05todos\x02\x04\x00\xdf\xc1\xa5\xe9\x0f\x00\x02fo\x84\xdf\xc1\xa5\xe9\x0f\x02\x01oF\xdf\xc1\xa5\xe9\x0f\x01\x04some\x0c"attributes"\xc6\xdf\xc1\xa5\xe9\x0f\x02\xdf\xc1\xa5\xe9\x0f\x03\x04some\x04null\x87\xdf\xc1\xa5\xe9\x0f\x00\x02\x04\x00\xdf\xc1\xa5\xe9\x0f\x06\x03foo\x00',
 'sv': b'\x01\xdf\xc1\xa5\xe9\x0f\n',
 'last_sv': b'\x01\xdf\xc1\xa5\xe9\x0f\x06',
 'partial_diff': b'\x01\x02\xdf\xc1\xa5\xe9\x0f\x06\x87\xdf\xc1\xa5\xe9\x0f\x00\x02\x04\x00\xdf\xc1\xa5\xe9\x0f\x06\x03foo\x00'}

<IPython.core.display.Javascript object>

### Load TodoCRDT at different snapshots

Notice when we load the snapshot (the first snapshot with just YText(foo), not the second snapshot with [foo, bar]), we only see one delta event: inserting the fully formed YText into the YArray.

y-pydantic needs work to read in that text when it doesn't observe updates to an empty YText object, and I don't see any way to introspect the attached attributes on the YText object.

In [51]:
new = TodoCRDT()
new.doc.begin_transaction().apply_v1(snapshot["full_diff"])
new

[YArrayEvent(target=['foo'], delta=[{'insert': [YText(foo)]}], path=[])]


<Todos ['']>

<IPython.core.display.Javascript object>

In [52]:
new.todos.yarray

YArray(['foo'])

<IPython.core.display.Javascript object>

In [53]:
new.todos.model

ArrayModel(items=[<TextBinding >], deleted=[])

<IPython.core.display.Javascript object>

In [54]:
new.todos.yarray[0]

YText(foo)

<IPython.core.display.Javascript object>

In [55]:
new.todos.model.items[0]

<TextBinding >

<IPython.core.display.Javascript object>